# Environment Setup
First enter into virtualenv. Do below in terminal:

In [ ]:
# virtualenv venv
# source venv/bin/activate
# pip install -r requirements.txt
# python -m easy_install carla-0.9.6-py3.5-linux-x86_64.egg

# Select venv/bin/python in vscode

# Change ipython kernel to venv/bin/python when prompted

Then setup PYTHONPATH as below:

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('.'))
import oatomobile

# Dataset
## Option 1: Download Datset from OATML

In [ ]:
from oatomobile.datasets import CARLADataset
output_dir = "data-oatml"
dataset = CARLADataset("raw")
dataset.download_and_prepare(output_dir)

In [ ]:
from oatomobile.datasets import CARLADataset
output_dir = "data-oatml"
dataset = CARLADataset("examples")
dataset.download_and_prepare(output_dir)

In [ ]:
from oatomobile.datasets import CARLADataset
output_dir = "data-oatml"
dataset = CARLADataset("processed")
dataset.download_and_prepare(output_dir)

## Option 2: Collect from Carla
To confirm.

In [ ]:
from oatomobile.datasets import CARLADataset
town = "Town01" # Options: Town01, Town02, Town03, Town04, Town05
town2 = "Town02" # for validation examples?
raw_dir = "data/raw" 
processed_dir = "data/processed" 
examples_dir = "data/examples" 
vehicles = 10
pedestrians = 10
render = True
dataset = CARLADataset("raw")
dataset.collect(town=town, output_dir=raw_dir, num_vehicles=vehicles, 
    num_pedestrians=pedestrians, render=render)
dataset.process(dataset_dir=raw_dir, output_dir=processed_dir)
# Not sure what examples are yet
dataset.collect(town=town2, output_dir=examples_dir, num_vehicles=vehicles, 
    num_pedestrians=pedestrians, render=render, num_steps=50)

# Training

The options are dim and cil. 

## Train the dim Imitative Model

In [ ]:
!PYTHONPATH="." python oatomobile/baselines/torch/dim/train.py \
    --dataset_dir=data-oatml/processed \
    --output_dir=data-oatml/model-dim \
    --num_epochs=50

## Train the cil Behavioural Model

In [ ]:
!PYTHONPATH="." python oatomobile/baselines/torch/cil/train.py \
    --dataset_dir=data/dataset \
    --output_dir=data/model/cil \
    --num_epochs=50

# Inference

Let's drive (inferernce)!

## Inference from dim imitative model

In [ ]:
# Imitation-learners.
import torch
import oatomobile.baselines.torch
import oatomobile
from oatomobile.envs import CARLAEnv

ckpt = "data-oatml/model/dim/ckpts/model-640.pt"
town = "Town02"

model = oatomobile.baselines.torch.ImitativeModel()
model.load_state_dict(torch.load(ckpt))

# Initializes a CARLA environment.
environment = CARLAEnv(town=town)
# Makes an initial observation.
observation = environment.reset()
done = False

agent = oatomobile.baselines.torch.DIMAgent(
  environment=environment,
  model=model,
  )

while not done:
  action = agent.act(observation)
  observation, reward, done, info = environment.step(action)
  # Renders interactive display.
  environment.render(mode="human")

# # Book-keeping: closes
environment.close()

## Inference from cil bahivior model

In [ ]:
# Imitation-learners.
import torch
import oatomobile.baselines.torch
import oatomobile
from oatomobile.envs import CARLAEnv

town = "Town02"
ckpt = "data-oatml/model/dim/ckpts/model-600.pt" # Paths to the model checkpoints.

model = oatomobile.baselines.torch.ImitativeModel()
model.load_state_dict(torch.load(ckpt))

# Initializes a CARLA environment.
environment = CARLAEnv(town=town)
# Makes an initial observation.
observation = environment.reset()
done = False

agent = oatomobile.baselines.torch.CILAgent(
  environment=environment,
  model=model
  )

while not done:
  action = agent.act(observation)
  observation, reward, done, info = environment.step(action)
  # Renders interactive display.
  environment.render(mode="human")

# # Book-keeping: closes
environment.close()

## Inference from rip ensemble model

Can we handle out of distribution?

In [ ]:
# Imitation-learners.
import torch
import oatomobile.baselines.torch
import oatomobile
from oatomobile.envs import CARLAEnv

town = "Town02"
ckpts = [ \
  "data-oatml/model/dim/ckpts/model-100.pt",
  "data-oatml/model/dim/ckpts/model-200.pt",
  "data-oatml/model/dim/ckpts/model-300.pt",
  "data-oatml/model/dim/ckpts/model-400.pt", 
  ] # Paths to the model checkpoints.
models = [oatomobile.baselines.torch.ImitativeModel() for _ in range(4)]

for model, ckpt in zip(models, ckpts):
  model.load_state_dict(torch.load(ckpt))

# Initializes a CARLA environment.
environment = CARLAEnv(town=town)
# Makes an initial observation.
observation = environment.reset()
done = False

agent = oatomobile.baselines.torch.RIPAgent(
  environment=environment,
  models=models,
  algorithm="WCM",
  )

while not done:
  action = agent.act(observation)
  observation, reward, done, info = environment.step(action)
  # Renders interactive display.
  environment.render(mode="human")

# # Book-keeping: closes
environment.close()

# Analysis

Open tensorboard

In [ ]:
!tensorboard --logdir data/model/dim/logs/